## Item 23: Accept Functions for Simple Interfaces Instead of Classes

* An Application Programming Interface (API) is a computing interface that defines `interactions` between multiple software intermediaries.
* Many of Python's built-in APIs allow you to customize behavior by passing in a function.
* These `hooks` are used by APIs to call back your code while they execute.

In [ ]:
from collections import defaultdict

* Example

In [ ]:
names = ["Socrates", "Archimedes", "Plato", "Aristotle"]
names.sort()  # (*, key=None, reverse=False)

In [ ]:
names

In [ ]:
names.sort(key=lambda x: len(x))

In [ ]:
names

* In Python, many hooks ae just stateless functions with well-defined arguments and return values.
* Functions are ideal for hooks because they are easier to describe and simpler to define than classes.

### Review defaultdict

https://docs.python.org/3/library/collections.html#collections.defaultdict

* Using `list` as the default_factory

In [ ]:
s = [("yellow", 1), ("blue", 2), ("yellow", 3), ("blue", 4), ("red", 5)]
d = defaultdict(list)

for k, v in s:
    d[k].append(v)

In [ ]:
d.items()

In [ ]:
sorted(d.items())

* Using dict.setdefault()

In [ ]:
d = {}

for k, v in s:
    d.setdefault(k, []).append(v)

In [ ]:
sorted(d.items())

* Setting the default_factory to `int`
    * supply a default count of zero. The increment operation then builds up the count for each letter

In [ ]:
s = "mississippi"
d = defaultdict(int)

for k in s:
    d[k] += 1

In [ ]:
sorted(d.items())

* Setting the default_factory to `set`
    * makes the defaultdict useful for building a dictionary of sets

In [ ]:
s = [("yellow", 1), ("blue", 2), ("yellow", 3), ("blue", 4), ("red", 5)]
d = defaultdict(set)

for k, v in s:
    d[k].add(v)

In [ ]:
sorted(d.items())

### Define a hook

* Customize the behavior of the `defaultdict` class.

    * Supplying functions like `log_missing` makes APIs easy to build and test.
    * It separates side effects from deterministic behavior.

In [ ]:
def log_missing():
    print(f"{key} added")
    return 0

In [ ]:
current = {"green": 12, "blue": 3}
increments = [
    ("red", 5),
    ("blue", 17),
    ("orange", 9),
]

In [ ]:
result = defaultdict(log_missing, current)
print(dict(result))

In [ ]:
for (key, amount) in increments:
    result[key] += amount   

In [ ]:
print(dict(result))

* Define a helper function that passes to defaultdict to count the total number of keys that were missing.
    * Using a stateful closure

In [ ]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # stateful closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for (key, count) in increments:
        result[key] += count
    
    return result, added_count   

In [ ]:
result, count = increment_with_report(current, increments)
assert count == 2

In [ ]:
dict(result)

In [ ]:
count

### Problem
    * Defining a closure for stateful hooks is that it's harder to read than the stateless function example.
    
  
* Another approach
    * Define a small class that encapsulates the state you want to track.

In [ ]:
class CountMissing:
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

* In Python, thanks to first-class functions, you can reference the `CountMissing.missing` directly on an object and pass it to `defaultdict` as the default value hook.

In [ ]:
counter = CountMissing()
result = defaultdict(counter.missing, current)  # method ref

for key, amount in increments:
    result[key] += amount
    
assert counter.added == 2

* Not clear?
    * What the purpose of the CounterMissing class?
    * Who constructs a CountMissing object?
    
* To clarify, Python allows classes to define the `__call__` special methods.
    * `__call__` allows an object to be called just like a function.
    * It causes the callable built-in function to return `True` for such an instance.

### Better 

In [ ]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0

In [ ]:
counter = BetterCountMissing()
counter()

* Return True if the object argument appears callable, False if not.

https://docs.python.org/3/library/functions.html#callable

In [ ]:
assert callable(counter)

In [ ]:
counter()

* Use a BetterCountMissing instance as the default value hook for a defauttdict to track the number of missing keys that were added.
* It provides a strong hint that the goal of the class is to act as a stateful closure.

In [ ]:
counter = BetterCountMissing()
result = defaultdict(counter, current)  # relies on __call__

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

* `defaultdict` still has no view into what's going on when you use `__call_`_.
* All that `defaultdict` requires is a function for the default value hook.

### Things to Remember

* Instead of defining and instantiating classes, functions are often all you need for simple interface betweren components in Python.
* References to functions and methods in Python are first class, meaning they can be used in expressions like any other type.
* The `__call__` special method enables instances of a class to be called like plain Python functions.
* When you need a function to maintain state, consider defining a class that provides the `__call__` method instead of defining a stateful closure.